# David Brookes
# April 2022

# Two Factor ANOVA (Analysis of Variance)

# Task achieved using:
# 1. Simple python commands
# 2. Library software

The analysis of variance using one factor can be extended to two (or more) factors.
(See code the 'ANOVA One Factor').    \
E.g. Yields per acre of four different plant crops grown on lots treated with three different types of fertilizer. In this case Yield is the dependent continuous variable, and crops and fertilizer are the independent categorical variables (factors).   

Note that it is assumed that only one measurement is taken for the Yield (in contrast to the example given in the ANOVA One Factor code where there were replications - i.e. several measurements).


 

(Theory and examples taken from the book Probability and Statistics - Schaum's Outline Series).

ANOVA test assumptions:

The ANOVA test has important assumptions that must be satisfied in order for the associated p-value to be valid.

The samples are independent.

Each sample is from a normally distributed population.

The population standard deviations of the groups are all equal. This property is known as homoscedasticity.

In [1]:
fertilizer = ['A', 'B', 'C']
crop = ['I', 'II', 'III', 'IV']

In [2]:
import numpy as np

x = np.array([[4.5, 6.4, 7.2, 6.7], [8.8, 7.8, 9.6, 7.0], [5.9, 6.8, 5.7, 5.2]]) 
print(x)

[[4.5 6.4 7.2 6.7]
 [8.8 7.8 9.6 7. ]
 [5.9 6.8 5.7 5.2]]


In [3]:
a, b = x.shape

print((a,b))

(3, 4)


In [4]:
fert_index = fertilizer.index('C')
crop_index = crop.index('crop_IV')

print(x[fert_index, crop_index])

5.2


In [5]:
# Find the fertilizer (row) means.

x_row_mean = np.mean(x, axis=1)
print(x_row_mean)

[6.2 8.3 5.9]


In [6]:
# Find the crop (column) means.

x_col_mean = np.mean(x, axis=0)
print(x_col_mean)

[6.4 7.  7.5 6.3]


In [7]:
# Find the grand mean.

x_grand_mean = np.mean(x)
print(x_grand_mean)

6.800000000000001


In [8]:
# Find the total variation.
# (Using the efficient numpy array iterator np.nditer).
v = 0.0
for el in np.nditer(x):
    v += (el-x_grand_mean)**2
print(v)

23.08


In [9]:
# Find the variation between fertilizers (rows).
# (Using the efficient numpy array iterator np.nditer).
v_r = 0.0
for el in np.nditer(x_row_mean):
    v_r += (el-x_grand_mean)**2
v_r *= b
print(v_r)

13.680000000000012


In [10]:
# Find the variation between crops (columns).
# (Using the efficient numpy array iterator np.nditer).
v_c = 0.0
for el in np.nditer(x_col_mean):
    v_c += (el-x_grand_mean)**2
v_c *= a
print(v_c)

2.8199999999999976


In [11]:
# Find the variation due to error (rows).
# (Using the efficient numpy array iterator np.nditer).
# numpy arrays x and x__row_mean can be iterated simultaneously.

v_e = 0.0
for el, m1, m2 in np.nditer([x, x_row_mean.reshape(a,1), x_col_mean.reshape(1,b)]):
    v_e += (el - m1- m2 + x_grand_mean)**2

print(v_e)

6.58


In [12]:
# A quick way to work out the variation due to error is as follows.
# (This acts as a check for the above calculation)

v_e = v - v_r - v_c
print(v_e)

6.5799999999999885


There are two hypotheses that we may want to test   \
H0(1): The fertilizer (row) means are equal.   \
H0(1): The crop (column) means are equal.     

In [13]:
# Find an unbiased estimate of the population variance using: 
# the variation due to error.

# Note that this is the best estimate of the variance regardless of
# whether either of the null hypotheses is true or not.

var_e = v_e/((a-1)*(b-1))
print(var_e)

1.0966666666666647


In [14]:
# Find an unbiased estimate of the population variance using: 
# the variation between fertilizers (rows), under the null hypothesis
# that all fertilizer (row) means are equal (i.e. H0(1) is true).

var_r = v_r/(a-1)
print(var_r)

6.840000000000006


In [15]:
# Find an unbiased estimate of the population variance using: 
# the variation between crops (columns), under the null hypothesis
# that all crop (column) means are equal (i.e. H0(2) is true).

var_c = v_c/(b-1)
print(var_c)


0.9399999999999992


In [16]:
# Find an unbiased estimate of the population variance using: 
# the total variation, under the null hypothesis
# that all fertilizer (row) means are equal, that all crop (column) means are equal (i.e. H0(1) and H0(2) are true).

var = v/(a*b-1)
print(var)


2.098181818181818


In [17]:
# Under the null hypothesis H0(1) of equal fertilizer (row) means the statistic
# var_r/var_e has an F distribution with (a-1), (a-1)(b-1) degrees of freedom. 
# This provides a test for the null hyothesis.

F1 = var_r/var_e

print(F1)

6.237082066869318


In [18]:
# To find the p value it is necessary to calculate areas
# under the F distribution curve.
# I'm going to use the SciPy library here to save time,
# rather performing numerical integration!

from scipy.stats import f
f_stat = F1
dof1 = a - 1
dof2 = (a-1)*(b-1)

# p-value for 1-sided test
print('p_value = ', 1 - f.cdf(abs(f_stat), dof1, dof2))

p_value =  0.034257791179503894


At the 0.05 level of signicance H0(1) can be rejected (since 0.034 < 0.05).   \
The fertilizer (row) means are not equal, and there is a difference in yield   \
due to the fertilizers used.

In [19]:
# Under the null hypothesis H0(2) of equal crop (column) means the statistic
# var_r/var_e has an F distribution with (b-1), (a-1)(b-1) degrees of freedom. 
# This provides a test for the null hyothesis.

F2 = var_c/var_e

print(F2)

0.857142857142858


In [20]:
# Calculate the corresponding p value.

f_stat = F2
dof1 = b - 1
dof2 = (a-1)*(b-1)

# p-value for 1-sided test
print('p_value = ', 1 - f.cdf(abs(f_stat), dof1, dof2))

p_value =  0.5121845972219611


At the 0.05 level of signicance H0(2) cannot be rejected (since 0.512 > 0.05).
The crop (column) means are equal, and there is a no difference in yield   \
due to the crops used.

# Performing ANOVA Two Factor Test with statsmodels.api Library

In [21]:
# Note that the scipy library does not appear to have a Two Factor ANOVA function.

In [22]:
import numpy as np
import pandas as pd

In [23]:
# Create a dataframe.
df = pd.DataFrame(columns = ['Fertilizer', 'Crop', 'Yield'])

print(df)

Empty DataFrame
Columns: [Fertilizer, Crop, Yield]
Index: []


In [24]:
# Populate the dataframe with data from numpy array x.
for findex in range(len(fertilizer)):
    for cindex in range(len(crop)):
        #print('findex:', findex,'cindex:', cindex )
        df.loc[len(df.index)] = [fertilizer[findex], crop[cindex], x[findex, cindex]]    
print(df)       

   Fertilizer      Crop  Yield
0           A    crop_I    4.5
1           A   crop_II    6.4
2           A  crop_III    7.2
3           A   crop_IV    6.7
4           B    crop_I    8.8
5           B   crop_II    7.8
6           B  crop_III    9.6
7           B   crop_IV    7.0
8           C    crop_I    5.9
9           C   crop_II    6.8
10          C  crop_III    5.7
11          C   crop_IV    5.2


In [25]:
# Importing libraries.
import statsmodels.api as sm
from statsmodels.formula.api import ols
  
# Performing Two Factor ANOVA.
#model = ols('Yield ~ C(Fertilizer) + C(Crop) + C(Fertilizer):C(Crop)', data=df).fit()
model = ols('Yield ~ C(Fertilizer) + C(Crop)', data=df).fit()
result = sm.stats.anova_lm(model, typ=2)

print(result)

               sum_sq   df         F    PR(>F)
C(Fertilizer)   13.68  2.0  6.237082  0.034258
C(Crop)          2.82  3.0  0.857143  0.512185
Residual         6.58  6.0       NaN       NaN
